In [ ]:
!wget https://cildata.crbs.ucsd.edu/ccdb//telescience/home/CCDB_DATA_USER.portal/P2043/Experiment_6835/Subject_6837/Tissue_6840/Microscopy_6843/MP6843_img_full.zip
!wget https://cildata.crbs.ucsd.edu/ccdb//telescience/home/CCDB_DATA_USER.portal/P2043/Experiment_6835/Subject_6837/Tissue_6840/Microscopy_6843/MP6843_seg.zip
!unzip -o -d ./images MP6843_img_full.zip
!unzip -o -d ./labels MP6843_seg.zip
!rm *.zip

In [ ]:
from os.path import join
from os import listdir
import numpy as np
from PIL import Image
import torch
from torch import nn, optim, float
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import matplotlib.pyplot as plt

In [ ]:
im_root, lb_root = "./images", "./labels"
images = [join(im_root, im) for im in sorted(listdir(im_root)) if 'w1' in im]
labels = [join(lb_root, lb) for lb in sorted(listdir(lb_root)) if '_01' in lb]

In [ ]:
class CellImages(Dataset):
    def __init__(self, images, labels):
        self.images, self.labels = images, labels
        self.transform = transforms.ToTensor()

    def __len__(self):
        return len(self.images)

    def __getitem__(self, ind):
        img = self.transform(Image.open(self.images[ind]))
        lbl = self.transform(Image.open(self.labels[ind]))
        return img, lbl

In [ ]:
train = DataLoader(CellImages(images, labels), batch_size=20)
for i, sample in enumerate(train):
    if i == 2:
        img, lbl = sample
        for ind in range(3):
            f, ax = plt.subplots(1,2,figsize=(10,10))
            ax[0].imshow(img[ind].numpy()[0])
            ax[1].imshow(lbl[ind].numpy()[0])
        plt.show()
        break